In [ ]:
%matplotlib inline
#這是juoyter notebook的magic word˙

import matplotlib
import matplotlib.pyplot as plt
from IPython import display

# iWildCam2020 revised3

In [ ]:
import os
#判斷是否在jupyter notebook上
def is_in_ipython():
    "Is the code running in the ipython environment (jupyter including)"
    program_name = os.path.basename(os.getenv('_', ''))

    if ('jupyter-notebook' in program_name or # jupyter-notebook
        'ipython'          in program_name or # ipython
        'jupyter' in program_name or  # jupyter
        'JPY_PARENT_PID'   in os.environ):    # ipython-notebook
        return True
    else:
        return False


#判斷是否在colab上
def is_in_colab():
    if not is_in_ipython(): return False
    try:
        from google import colab
        return True
    except: return False

#判斷是否在kaggke_kernal上
def is_in_kaggle_kernal():
    if 'kaggle' in os.environ['PYTHONPATH']:
        return True
    else:
        return False

if is_in_colab():
    from google.colab import drive
    drive.mount('/content/gdrive')

In [ ]:
os.environ['TRIDENT_BACKEND'] = 'pytorch'

if is_in_kaggle_kernal():
    os.environ['TRIDENT_HOME'] = './trident'
    
elif is_in_colab():
    os.environ['TRIDENT_HOME'] = '/content/gdrive/My Drive/trident'

#為確保安裝最新版 
!pip uninstall tridentx -y
!pip install '../input/trident/tridentx-0.7.5-py3-none-any.whl' --upgrade
import json
import copy
import numpy as np
#調用trident api
import trident as T
from trident import *
from trident.models import resnet,efficientnet

In [ ]:
import pandas as pd

with open('../input/iwildcam-2020-fgvc7/iwildcam2020_train_annotations.json') as json_file:
    train_data = json.load(json_file)


with open('../input/iwildcam-2020-fgvc7/iwildcam2020_test_information.json') as test_json_file:
    test_data = json.load(test_json_file)

    
df_train = pd.DataFrame({'id': [item['id'] for item in train_data['annotations']],
                         'category_id': [item['category_id'] for item in train_data['annotations']],
                         'image_id': [item['image_id'] for item in train_data['annotations']],
                         'location': [item['location'] for item in train_data['images']],
                         'file_name': [item['file_name'] for item in train_data['images']]})
df_test = pd.DataFrame({'image_id': [item['id'] for item in train_data['images']],
                         'location': [item['location'] for item in train_data['images']],
                         'file_name': [item['file_name'] for item in train_data['images']]})



df_train

In [ ]:
df_category_train=pd.DataFrame({'id': [item['id'] for item in train_data['categories']],
                         'name': [item['name'] for item in train_data['categories']],
                         'count': [item['count'] for item in train_data['categories']]})

df_category_test=pd.DataFrame({'id': [item['id'] for item in test_data['categories']],
                         'name': [item['name'] for item in test_data['categories']],
                         'count': [item['count'] for item in test_data['categories']]})

df_category_train=df_category_train.sort_values(['count'],ascending=False) 
print(df_category_train)
df_category_test=df_category_test.sort_values(['count'],ascending=False) 
print(df_category_test)

#基於標註檔，產生有在標註檔內的所有圖片的清單，進行去重複(set)、排序(sorted)以及轉換成清單(list)
animal_category_lists=list(sorted(set([item['category_id'] for item in train_data['annotations']])))


#進行篩選
df_category_train=df_category_train[df_category_train['id'].isin(animal_category_lists)]
print(df_category_train)

df_category_test=df_category_test[df_category_test['count']>0]
print(df_category_test)

animal_category_lists_train=[category_id.item() for category_id in df_category_train[['id']].to_numpy().astype(np.int64)]
print(animal_category_lists_train[:5])

animal_category_lists_test=[category_id.item() for category_id in df_category_test[['id']].to_numpy().astype(np.int64)]
print(animal_category_lists_test[:5])

#檢查是不是所有df_category_test數量不為零的動物都有出現在df_category_train的類別代號中
category_missing_list=[category_id for category_id in animal_category_lists_test if category_id not in animal_category_lists_train]
print(category_missing_list)



除了數量外，我們也要注意到，出現最多的動物以及出現最少的動物，其圖片數量相差了1.6萬倍，由於有許多動物只出現一次，為了避免包含只有出現過一次的動物的圖片因為數據增強或是其他因素意外的沒有呈現出該動物特徵而讓整個動物類別都無法識別，因此，我打算先把所有出現頻率超低的動物先列出來，然後將包含這些動物的圖片多複製幾分與原圖片集融合，這樣就比較不會有前述的問題。

In [ ]:
label2category=OrderedDict()
category2label=OrderedDict()
#產生能將category_id轉label的字典
for i in range(len(animal_category_lists_train)):
    category2label[animal_category_lists_train[i]]=i
    label2category[i]=animal_category_lists_train[i]

#建構出轉成標籤id
label_idxes=[category2label[item['category_id']] for item in train_data['annotations']]
image_pathes=['../input/iwildcam-2020-fgvc7/train/'+item['file_name'] for item in train_data['images']]

print('label_idxes',label_idxes[:5])
print('image_pathes',image_pathes[:5])
print('label2category',list(label2category.items())[:5])
print('category2label',list(category2label.items())[:5])

In [ ]:
rare_animals=OrderedDict()
common_animals=OrderedDict()
df_animal_frequency=df_train['category_id'].value_counts()

df_rare_animal_frequency=df_animal_frequency[df_animal_frequency<=10]
df_common_animal_frequency=df_animal_frequency[df_animal_frequency>1000]

for item in  df_rare_animal_frequency.iteritems() :
    rare_animals[item[0]]=item[1]
    
for item in  df_common_animal_frequency.iteritems() :
    common_animals[item[0]]=item[1]
    
print('rare_animals',len(rare_animals))
print('common_animals',len(common_animals))

In [ ]:
import glob
#透過glob所全部train資料夾中所有可用圖片
imgs=glob.glob('../input/iwildcam-2020-fgvc7/train/*.jpg')
print(len(imgs))
print(imgs[:5])

#將圖檔路徑去除資料夾部分後進行去重複
img_pathes=[img.split('/')[-1] for img in imgs]
img_pathes=list(sorted(set(img_pathes)))
print(len(img_pathes))
print(img_pathes[:5])


In [ ]:
rare_images=[]
rare_labels=[]
for i in range(len(image_pathes)):
    img=image_pathes[i]
    label=label_idxes[i]
    if label2category[label] in rare_animals:
        cnt=rare_animals[label2category[label]]
        for n in range(int(20.0/cnt)):
             rare_images.append(img) 
             rare_labels.append(label) 
        
print(len(rare_images))
print(len(rare_labels))
print(rare_images[:5])
print(rare_labels[:5])

In [ ]:

img_ds=ImageDataset(image_pathes+rare_images,symbol='image')
label_ds=LabelDataset(label_idxes+rare_labels,symbol='label')

#非常見動物的全部+常見動物基於頻率調整成為1000
sample_filter=lambda x:label2category[x[-1]] not in common_animals.key_list or random.random()<(1000.0/common_animals[label2category[x[-1]]])


#與Iterator構成data provider
data_provider=DataProvider(traindata=Iterator(data=img_ds,label=label_ds))

data_provider.image_transform_funcs=[
    Resize((224,224)),
    CLAHE(),
    RandomAdjustGamma(scale=(0.8,1.2)),#調整明暗
    RandomAdjustHue(scale=(-0.2,0.2)),#調整色相
    RandomAdjustSaturation(scale=(0.8,1.2)),#調整飽和度
    SaltPepperNoise(0.005, keep_prob=0.5),#加入胡椒鹽噪音
    RandomErasing(size_range=(0.05, 0.2), transparency_range=(0.4, 0.8), transparancy_ratio=1.0, keep_prob=0.5), #加入隨機擦去
    RandomTransformAffine(rotation_range=45, zoom_range=0.00, shift_range=0.00, shear_range=0.2, random_flip=0.15),#隨機仿射變換
    Normalize(127.5,127.5)] #標準化


data,labels=data_provider.next()
print(data.shape)
print(labels)

In [ ]:
%%time
data_provider.preview_images()

In [ ]:
from trident.models import efficientnet

net1=efficientnet.EfficientNetB0(pretrained=True,include_top=False,classes=len(animal_category_lists_train),input_shape=(3,224,224),freeze_features=True)
net1.model.add_module('last_conv',Conv2d_Block((3,3),num_filters=len(animal_category_lists_train),use_bias=False,activation=None, normalization='l2'))
cam=ShortCut(
    Identity(),
    Sequential(
    GlobalAvgPool2d(),
    Reshape((len(animal_category_lists_train),1,1)),
    Conv2d((1,1),num_filters=len(animal_category_lists_train),use_bias=False,activation=None)
    )
,mode='dot'
)

net1.model.add_module('cam',cam)
net1.model.add_module('aggregate1',Aggregation('sum',axis=2))
net1.model.add_module('aggregate2',Aggregation('sum',axis=3))
net1.model.add_module('reshape',Reshape((len(animal_category_lists_train))))
net1.model.add_module('sigmoid',Sigmoid())
net1.model.add_module('fc',Dense((len(animal_category_lists_train))))
net1.model.add_module('softmax',SoftMax(axis=-1,add_noise=True,noise_intensity=0.12))
net1.summary()

net1.model.block7a.trainable=True

if os.path.exists('./Models/revised3_net1.pth.tar'):
    net1.load_model('./Models/revised3_net1.pth.tar')
elif os.path.exists('../input/iwildcam2020-revised3/Models/revised3_net1.pth.tar'):
    net1.load_model('../input/iwildcam2020-revised3/Models/revised3_net1.pth.tar')


這次我們增加的一個新的網路，看起來跟net1幾乎一樣，但是在最後全連接層之前加入了L2NORM，最後全連接層也加入了weights_norm='l2'，以及不再加入Softmax，這到底是在做甚麼呢。基本上應該要跟後面的新的損失函數ArcMarginProductLoss放在一起看才會有意義。

In [ ]:
net2=efficientnet.EfficientNetB0(pretrained=True,include_top=False,classes=len(animal_category_lists_train),input_shape=(3,224,224),freeze_features=True)
net2.model.add_module('output_layer', 
    Sequential(
        Dropout(dropout_rate=0.4),
        Flatten(),
        Dense((512),use_bias=False),
    ))
net2.model.add_module('l2norm',L2Norm())
net2.model.add_module('fc',Dense((len(animal_category_lists_train)),use_bias=False,weights_norm='l2'))
net2.summary()
net2.model.block7a.trainable=True

if os.path.exists('./Models/revised3_net2.pth.tar'):
    net2.load_model('./Models/revised3_net2.pth.tar')
elif os.path.exists('../input/iwildcam2020-revised3/Models/revised3_net2.pth.tar'):
    net2.load_model('../input/iwildcam2020-revised3/Models/revised3_net2.pth.tar')


In [ ]:
from trident.models import visual_transformer
vit=visual_transformer.VisionTransformer_small(pretrained=True,input_shape=(3,224,224),patch_size=16,num_classes=len(animal_category_lists_train))


vit.model.blocks[10].trainable=True
vit.model.blocks[11].trainable=True
vit.summary()

if os.path.exists('./Models/revised3_vit.pth.tar'):
    vit.load_model('./Models/revised3_vit.pth.tar')
elif os.path.exists('../input/iwildcam2020-revised3/Models/revised3_vit.pth.tar'):
    vit.load_model('../input/iwildcam2020-revised3/Models/revised3_vit.pth.tar')


ArcMarginProductLoss是跟ARCFACE借來的概念。為什麼人臉識別模型可以處理全球70億人口的人臉識別?而且可能公司只索取我們一張照片就可以做人臉識別使用。那又是決策邊界的概念，要如何設置一個可以處理區辨70億人口人臉的決策邊界的，那唯一的方法就是要讓大家排排站在一個高維的「超球」，人跟人之間至少要間隔多少度以上的夾角，但是同個人的不同樣貌則是位於很窄的局部區域中。這樣就有機會用少量圖片處理大量人口。我們借這概念來到野生動物識別，有的動物數量很多，有的動物數量極少，我們藉由超球的概念來區辨動物們。

<img src='https://docs.google.com/uc?export=download&id=17OrPMDGXom4aVpWZRQm93RbhtEHxsVs1'/>


In [ ]:
class ArcMarginProductLoss(Layer):
    def __init__(self, scale=32.0, margin=0.50, easy_margin=False, name='ArcMarginProductLoss'):
        super(ArcMarginProductLoss, self).__init__()
        self._name=name
        self.scale = scale
        self.m = margin
        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)

        # make the function cos(theta+m) monotonic decreasing while theta in [0°,180°]
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin
    
        self.base_loss=CrossEntropyLoss(reduction='mean',auto_balance=True)


    def forward(self, output, target,**kwargs):
        # cos(theta)
        try:
            cosine=output
            # cos(theta + m)
            sine = sqrt(1.0 - pow(cosine, 2))
            phi = cosine * self.cos_m - sine * self.sin_m

            if self.easy_margin:
                phi = where(cosine > 0, phi, cosine)
            else:
                phi = where((cosine - self.th) > 0, phi, cosine - self.mm)

            one_hot = zeros_like(cosine,requires_grad=True)
            one_hot.scatter(1, target.view(-1, 1), 1)

            output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
            output = output * self.scale
        except Exception as e:
            print(e)
            PrintException()

        loss = self.base_loss(output, target)
        return loss

In [ ]:
net1.with_optimizer(optimizer=AdaBelief,lr=5e-4,betas=(0.9, 0.999),gradient_centralization='all')\
.with_loss(CrossEntropyLoss(auto_balance=True,label_smooth=True))\
.with_loss(FocalLoss,loss_weight=0.5)\
.with_metric(accuracy,name='accuracy')\
.with_metric(accuracy,topk=5,name='top5_accuracy',print_only=True)\
.with_regularizer('l2',reg_weight=5e-5)\
.adjust_learning_rate_scheduling(1000,unit='batch',new_value=5e-4)\
.with_accumulate_grads(4)\
.with_callbacks(MixupCallback(alpha= 1,loss_criterion=CrossEntropyLoss,loss_weight=0.5))\
.with_learning_rate_scheduler(StepLR(frequency=1000,unit='batch',gamma=0.5))\
.with_model_save_path('./Models/revised3_net1.pth')\
.with_automatic_mixed_precision_training()


net2.with_optimizer(optimizer=AdaBelief,lr=5e-4,betas=(0.9, 0.999),gradient_centralization='all')\
.with_loss(ArcMarginProductLoss(scale=32.0, margin=0.50, easy_margin=False)) \
.with_metric(accuracy,name='accuracy')\
.with_metric(accuracy,topk=5,name='top5_accuracy',print_only=True)\
.with_regularizer('l2',reg_weight=5e-5)\
.adjust_learning_rate_scheduling(1000,unit='batch',new_value=5e-4)\
.with_accumulate_grads(4)\
.with_grad_clipping(3)\
.with_learning_rate_scheduler(StepLR(frequency=1000,unit='batch',gamma=0.5))\
.with_model_save_path('./Models/revised3_net2.pth')\
.with_automatic_mixed_precision_training()


#transformer一樣也是可以使用mixup的
vit.with_optimizer(optimizer=AdamW,lr=5e-4,betas=(0.9, 0.999),gradient_centralization='all',warmup=300)\
.with_loss(ArcMarginProductLoss(scale=32.0, margin=0.80, easy_margin=False)) \
.with_metric(accuracy,name='accuracy')\
.with_metric(accuracy,topk=5,name='top5_accuracy',print_only=True)\
.with_regularizer('l2',reg_weight=5e-5)\
.with_accumulate_grads(4)\
.with_grad_clipping(3)\
.with_learning_rate_scheduler(StepLR(frequency=1000,unit='batch',gamma=0.5))\
.with_model_save_path('./Models/revised3_vit.pth')\
.with_automatic_mixed_precision_training()


#如果兩個模型都做mixup，批次設定到64，gpu會爆掉
plan=TrainingPlan()\
    .add_training_item(net1,name='net1')\
    .add_training_item(net2,name='net2')\
    .add_training_item(vit,name='vit')\
    .with_data_loader(data_provider)\
    .with_batch_size(24)\
    .print_gradients_scheduling(200,unit='batch') \
    .print_progress_scheduling(10,unit='batch') \
    .display_loss_metric_curve_scheduling(200)\
    .save_model_scheduling(100,unit='batch')


plan.only_steps(num_steps=10000, collect_data_inteval=5)


In [ ]:
bad_images=['86994b3e-21bc-11ea-a13a-137349068a90.jpg',
'882a533a-21bc-11ea-a13a-137349068a90.jpg',
'88a28616-21bc-11ea-a13a-137349068a90.jpg',
'88b99aae-21bc-11ea-a13a-137349068a90.jpg',
'89362ed4-21bc-11ea-a13a-137349068a90.jpg',
'8985bb98-21bc-11ea-a13a-137349068a90.jpg',
'89e09b26-21bc-11ea-a13a-137349068a90.jpg',
'8a804608-21bc-11ea-a13a-137349068a90.jpg',
'8b8e02a6-21bc-11ea-a13a-137349068a90.jpg',
'8b91394e-21bc-11ea-a13a-137349068a90.jpg',
'8cc46b6a-21bc-11ea-a13a-137349068a90.jpg',
'8d705d8a-21bc-11ea-a13a-137349068a90.jpg',
'8e930668-21bc-11ea-a13a-137349068a90.jpg',
'8e940310-21bc-11ea-a13a-137349068a90.jpg',
'8ea6a768-21bc-11ea-a13a-137349068a90.jpg',
'8fff9dc2-21bc-11ea-a13a-137349068a90.jpg',
'9044a3b8-21bc-11ea-a13a-137349068a90.jpg',
'920ee4c4-21bc-11ea-a13a-137349068a90.jpg',
'950ed288-21bc-11ea-a13a-137349068a90.jpg',
'9522d4fe-21bc-11ea-a13a-137349068a90.jpg',
'96bacf06-21bc-11ea-a13a-137349068a90.jpg',
'98552f5a-21bc-11ea-a13a-137349068a90.jpg',
'98da656c-21bc-11ea-a13a-137349068a90.jpg',
'9955d012-21bc-11ea-a13a-137349068a90.jpg']



test_imgs=glob.glob('../input/iwildcam-2020-fgvc7/test/*.jpg')
print(len(test_imgs))
test_imgs=[img_path for img_path in test_imgs if img_path.split('/')[-1] not in bad_images]
print(len(test_imgs))


img_ds=ImageDataset(test_imgs,symbol='image')
#請注意，要設定object_type=ObjectType.image_path，這樣就可以確保輸出為stype=np.string_的numpy array
imgpath_ds=ImageDataset(test_imgs,object_type=ObjectType.image_path,symbol='img_path')

#與Iterator構成data provider
#設定is_shuffle=False表示不隨機打亂
test_data_provider=DataProvider(traindata=Iterator(data=img_ds,label=imgpath_ds,is_shuffle=False))

#保留有意義的數據清洗
test_data_provider.image_transform_funcs=[
    Resize((224,224)),
    CLAHE(),
    Normalize(127.5,127.5)] #標準化


data,labels=test_data_provider.next()
print(data.shape)
print(labels)
print(labels[0].item())
test_data_provider.preview_images()

In [ ]:
f=open('../input/iwildcam-2020-fgvc7/sample_submission.csv','r',encoding='utf-8-sig')
rows=f.readlines()
#檢視一下大會給的提交範例
print(rows[:5])
print(rows[-5:])

submission_dict=OrderedDict()
for row in rows[1:]:
    cols=row.strip().split(',')
    #排除大會公告壞掉的圖片
    if  cols[0]+'.jpg' not in bad_images:
        #塞入應有的KEY，value先放None
        submission_dict[cols[0]]=None
print(len(submission_dict))

In [ ]:
#清除gpu快取
import torch
if is_gpu_available():
    torch.cuda.synchronize()
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
net1.eval()
net2.eval()
vit.eval()
#設定批次為64
test_data_provider.batch_size=64
#這步驟是將generator歸零
test_data_provider.traindata.batch_sampler.reset()


for i,(img,img_path) in tqdm(enumerate(test_data_provider)):
    out1=net1(img)
    out2=net2(img)
    out3=vit(img)
    #基於輸出平均值
    out4=np.argmax((out1+1.5*out2+out3)/3.5,axis=1)
    #基於兩個模型評估的最大機率
    #out3=np.argmax(np.where(out1>out2,out1,out2),axis=1)
    for k in range(len(out4)):
        _,image_id,_=split_path(img_path[k].item())
        if i==0 and k==0:
            print(image_id)
        #將預測的標籤透過label2category轉回大會的類別
        category_id=label2category[out4[k]]
        if image_id in submission_dict and image_id+'.jpg' not in bad_images:
            submission_dict[image_id]=category_id
        #當submission_dict的值不再有空值則停止
        if None not in submission_dict.value_list:
            break
  

In [ ]:
submission_rows=['Id,Category\n']
for k,v in submission_dict.items():
    submission_rows.append('{0},{1}\n'.format(k,v))
#確認提交檔筆數
print(len(submission_rows))
#寫入提交檔
make_dir_if_need('./results')
with open('./results/submission.csv','w',encoding='utf-8-sig') as f:
    f.writelines(submission_rows)
#再讀取一次確認何大會給的範例格式是否一致
fr=open('./results/submission.csv','r',encoding='utf-8-sig')
rows=fr.readlines()
print(rows[:3])

In [ ]:
api_token={"username":"your_username","key":"your_token"} #請換成你自己的kaggle認證#請換成你自己的kaggle認證
import json
import zipfile
import os

if not os.path.exists("/root/.kaggle"):
    os.makedirs("/root/.kaggle")
 
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json
 
if not os.path.exists("/kaggle"):
    os.makedirs("/kaggle")
!kaggle competitions submit -c iwildcam-2020-fgvc7 -f './results/submission.csv' -m 'transformer/CAM_EfficientNetB0 ensembles'